# Tutorial 2: Analyzing Statistics and Variogram of the Ice Stream Data

### First load data as before

In [1]:
# load compiled bed elevation measurements
df = pd.read_csv('')

In [ ]:
# create a grid of x and y coordinates
x_uniq = np.unique(df.X)
y_uniq = np.unique(df.Y)

xmin = np.min(x_uniq)
xmax = np.max(x_uniq)
ymin = np.min(y_uniq)
ymax = np.max(y_uniq)

cols = len(x_uniq)
rows = len(y_uniq)

resolution = 1000

xx, yy = np.meshgrid(x_uniq, y_uniq)

In [ ]:
# load other data
velx, vely, velxerr, velyerr, fig = Topography.load_vel_measures('../../Data/antarctica_ice_velocity_450m_v2.nc', xx, yy)
dhdt, fig = Topography.load_dhdt('../Data/ANT_G1920_GroundedIceHeight_v01.nc',xx,yy,interp_method='linear',begin_year = 2013,end_year=2015,month=7)
smb, fig = Topography.load_smb_racmo('../Data/SMB_RACMO2.3p2_yearly_ANT27_1979_2016.nc', xx, yy, interp_method='spline',time=2014)
bm_mask, bm_source, bm_bed, bm_surface, bm_errbed, fig = Topography.load_bedmachine('../../Data/BedMachineAntarctica-v3.nc', xx, yy)

Now, it is the time to analyze the compiled radar data. How will you evaluate the roughness in this region? (hint: variogram)

In [ ]:
# find variograms
df_bed = df.copy()
df_bed = df_bed[df_bed["bed"].isnull() == False]
data = df_bed['bed'].values.reshape(-1,1)
coords = df_bed[['X','Y']].values
roughness_region_mask = (df_bed['bedmachine_mask'].values)==2

nst_trans, Nbed_radar, varios, fig = MCMC.fit_variogram(data, coords, roughness_region_mask, maxlag=70000, n_lags=70)

In addition, since we have the velocity data, it will also be nice to know where the high velocity region is, remembering that the mass conservation technique is best applied in high velocity region

In [ ]:
# calculate high velocity region
ocean_mask = (bm_mask == 0) | (bm_mask == 3)
grounded_ice_mask = (bm_mask == 2)
highvel_mask = Topography.get_highvel_boundary(velx, vely, 50, grounded_ice_mask, ocean_mask, 3000, xx, yy)

Let's generate a SGS bed for the entire region

In [ ]:
sgs_bed = 

SGS only generate a realization of the bed topography according to the known radar measurements. However, it might ignored something else. The thickness of the ice can be calculated as ice surface elevation minus bed elevation. From BedMachine, we know where the land is ice-free and where is covered by ice. However, SGS might generate bed such that it 'extrude' out of ice in the supposed grounded ice region. Let's fix that quickly

In [ ]:
thickness = bm_surface - sgs_bed
sgs_bed = np.where((thickness<=0)&(bm_mask==2), bm_surface-1, sgs_bed)

There are other things we would like to record before begin the MCMC chains

In [ ]:
cond_bed = df['bed'].values.reshape(xx.shape)
data_mask = ~np.isnan(cond_bed)